In [1]:
# 1. 克隆 LLaMA-Factory 仓库
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory

# 2. 安装依赖 (这一步可能需要几分钟)
!pip install -e ".[torch,metrics]"
!pip install bitsandbytes

# 3. 登录 HuggingFace (可选，为了下载某些受限模型，一般 Qwen 不需要)
# !huggingface-cli login

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 614, done.
remote: Counting objects: 100% (614/614), done.
remote: Compressing objects: 100% (455/455), done.
remote: Total 614 (delta 150), reused 378 (delta 101), pack-reused 0 (from 0)
Receiving objects: 100% (614/614), 5.24 MiB | 17.76 MiB/s, done.
Resolving deltas: 100% (150/150), done.
/content/LLaMA-Factory
Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 26.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 43.9 MB/s eta 0:00:00


In [2]:
import json

# 读取原本的 dataset_info.json
with open('data/dataset_info.json', 'r', encoding='utf-8') as f:
    dataset_info = json.load(f)

# 添加 Raymond 的配置
dataset_info['raymond_data'] = {
    "file_name": "raymond.json",  # 刚才上传的文件路径
    "formatting": "sharegpt",        # 你的数据格式属于 sharegpt 类型
    "columns": {
        "messages": "messages"       # 告诉框架去读 messages 字段
    }
}

# 保存回去
with open('data/dataset_info.json', 'w', encoding='utf-8') as f:
    json.dump(dataset_info, f, indent=2, ensure_ascii=False)

print("Raymond 数据集注册成功！")

Raymond 数据集注册成功！


In [ ]:
# 1. 确保在正确的目录
%cd /content/LLaMA-Factory

# 2. 启动 Web UI (使用新版命令)
!export GRADIO_SHARE=1 && llamafactory-cli webui

/content/LLaMA-Factory
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape sequence '\s'
  re_skip_default = re.compile("(\r\n|\s)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/finalseg/__init__.py:78: SyntaxWarning: invalid escape sequence '\.'
  re_skip = re.compile("([a-zA-Z0-9]+(?:\.\d+)?%?)")
Visit http://ip:port for Web UI, e.g., http://127.0.0.1:7860
* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://122536b96cfbf07c46.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os
import sys

# 1. 自动寻找最新的模型权重路径
save_root = "/content/LLaMA-Factory/saves"
# 注意：这里请确认你的底座模型路径，通常是 Qwen/Qwen2.5-7B-Instruct
base_model_path = "Qwen/Qwen2.5-7B-Instruct"
adapter_path = None

# 遍历目录寻找 checkpoint
if os.path.exists(save_root):
    checkpoints = []
    for root, dirs, files in os.walk(save_root):
        for d in dirs:
            if d.startswith("checkpoint-"):
                full_path = os.path.join(root, d)
                try:
                    step = int(d.split("-")[-1])
                    checkpoints.append((step, full_path))
                except ValueError:
                    continue

    if checkpoints:
        # 找到步数最大的那个
        checkpoints.sort(key=lambda x: x[0], reverse=True)
        adapter_path = checkpoints[0][1]
        print(f"✅ 找到最新模型权重: {adapter_path}")
    else:
        print("❌ 未找到 checkpoint，请确认训练是否至少跑完了一次保存步骤。")
else:
    print("❌ saves 目录不存在，请先进行训练。")

# 2. 如果找到了，启动独立聊天界面
if adapter_path:
    print("🚀 正在启动 Raymond 专属聊天室...")
    print("⚠️ 注意：进入网页后，请手动在左侧 'System Prompt' 框中输入：")
    print("   You are Raymond, a humorous chatter with a unique sarcastic style.")

    # 构造启动命令 (去掉了报错的 --system_prompt)
    cmd = (
        f"llamafactory-cli webchat "
        f"--model_name_or_path {base_model_path} "
        f"--adapter_name_or_path {adapter_path} "
        f"--template qwen "
        f"--finetuning_type lora "
        f"--quantization_bit 4 "
    )

    # 运行命令
    get_ipython().system(f"export GRADIO_SHARE=1 && {cmd}")

✅ 找到最新模型权重: /content/LLaMA-Factory/saves/Qwen2.5-7B-Instruct/lora/train_2026-02-11-09-48-53/checkpoint-100
🚀 正在启动 Raymond 专属聊天室...
⚠️ 注意：进入网页后，请手动在左侧 'System Prompt' 框中输入：
   You are Raymond, a humorous chatter with a unique sarcastic style.
Visit http://ip:port for Web UI, e.g., http://127.0.0.1:7860
[INFO|configuration_utils.py:667] 2026-02-11 11:51:36,743 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2.5-7B-Instruct/snapshots/a09a35458c702b33eeacc393d103063234e8bc28/config.json
[INFO|configuration_utils.py:739] 2026-02-11 11:51:36,746 >> Model config Qwen2Config {
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "dtype": "bfloat16",
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 3584,
  "initializer_range": 0.02,
  "intermediate_size": 18944,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    